<a href="https://colab.research.google.com/github/Milsy18/m18-model/blob/main/01_data_ingestion_and_alignment_ipynb1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
%%bash
cd /content/m18-model   # adjust if your project lives elsewhere
rm -rf .git             # delete any existing Git history
echo "✅ Removed any existing .git folder"


✅ Removed any existing .git folder


In [24]:
# Cell 1: clone and folder setup
import os
from pathlib import Path

# 1. Clone your GitHub repo (or pull latest if it already exists)
REPO_URL   = "https://github.com/YourUserName/m18-model.git"
PROJECT_ROOT = Path("/content/m18-model")

if not PROJECT_ROOT.exists():
    # first-time clone
    os.system(f"git clone {REPO_URL} {PROJECT_ROOT}")
else:
    # subsequent runs: update
    os.chdir(PROJECT_ROOT)
    os.system("git pull")

# 2. Create subfolders if they aren’t already present
for sub in ["data/raw", "data/processed", "modules", "outputs"]:
    (PROJECT_ROOT / sub).mkdir(parents=True, exist_ok=True)

# 3. Change working directory so all relative paths resolve
os.chdir(PROJECT_ROOT)
print("Working in", PROJECT_ROOT)


Working in /content/m18-model


In [25]:
# Cell 2: Data ingestion, epoch‐to‐datetime conversion & alignment
import pandas as pd
from pathlib import Path

# 1) Paths
RAW_DIR  = Path('/content')
PROC_DIR = Path('data/processed')
PROC_DIR.mkdir(parents=True, exist_ok=True)

# 2) Exact filenames as you uploaded them
file_map = {
    'BTC_D'  : 'CRYPTOCAP_BTC.D, 1D (1).csv',
    'TOTAL'  : 'CRYPTOCAP_TOTAL, 1D (1).csv',
    'TOTAL3' : 'CRYPTOCAP_TOTAL3, 1D (1).csv',
    'USDT_D' : 'CRYPTOCAP_USDT.D, 1D.csv',
}

market_dfs = {}
for name, fname in file_map.items():
    path = RAW_DIR / fname
    print(f"Loading {name} from {path}")
    df = pd.read_csv(path)

    #  └─ convert UNIX seconds → pandas DateTime index
    df['Date'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('Date', inplace=True)
    df.drop(columns=['time'], inplace=True)

    print(f"  ↳ {name}: dates {df.index.min().date()} → {df.index.max().date()}, shape {df.shape}")
    market_dfs[name] = df

# 3) Align all four on their intersection of dates
common_idx = market_dfs['BTC_D'].index
for df in market_dfs.values():
    common_idx = common_idx.intersection(df.index)
print(f"\nAligned date‐index length: {len(common_idx)}")

for name in market_dfs:
    market_dfs[name] = market_dfs[name].loc[common_idx]

# 4) Concatenate and save
aligned = pd.concat(market_dfs, axis=1)
aligned.to_csv(PROC_DIR / 'market_regimes_aligned.csv', index=True)
print(f"Saved aligned regimes to → {PROC_DIR/'market_regimes_aligned.csv'}")


Loading BTC_D from /content/CRYPTOCAP_BTC.D, 1D (1).csv
  ↳ BTC_D: dates 2020-01-01 → 2025-06-07, shape (1985, 5)
Loading TOTAL from /content/CRYPTOCAP_TOTAL, 1D (1).csv
  ↳ TOTAL: dates 2020-01-01 → 2025-06-07, shape (1985, 5)
Loading TOTAL3 from /content/CRYPTOCAP_TOTAL3, 1D (1).csv
  ↳ TOTAL3: dates 2020-01-01 → 2025-06-07, shape (1985, 5)
Loading USDT_D from /content/CRYPTOCAP_USDT.D, 1D.csv
  ↳ USDT_D: dates 2020-01-01 → 2025-06-07, shape (1985, 5)

Aligned date‐index length: 1985
Saved aligned regimes to → data/processed/market_regimes_aligned.csv


In [26]:
%%bash
# Configure your author name & email for Git
git config --global user.name "Misly18"
git config --global user.email "millarmike1@gmail.com"


In [27]:
# Properly read the two header rows into a MultiIndex
import pandas as pd

df = pd.read_csv(
    'data/processed/market_regimes_aligned.csv',
    header=[0,1],    # read rows 0 & 1 as a two‐level header
    index_col=0,     # first column is the date index
    parse_dates=True # parse that index as datetime
)
df.head()


BTC_D                                                 \
                 open       high        low      close        Volume   
Date                                                                   
2020-01-01  70.065894  70.144563  69.741007  69.921538  1.858042e+10   
2020-01-02  69.956957  70.129144  69.812534  69.920361  2.081380e+10   
2020-01-03  69.933572  70.185528  69.737601  69.955510  2.802447e+10   
2020-01-04  69.932530  70.091216  69.715515  69.875262  1.841875e+10   
2020-01-05  69.832141  69.997760  69.569826  69.704416  1.970856e+10   

                   TOTAL                                            \
                    open          high           low         close   
Date                                                                 
2020-01-01  1.854552e+11  1.880549e+11  1.847741e+11  1.861052e+11   
2020-01-02  1.860300e+11  1.867115e+11  1.784813e+11  1.801525e+11   
2020-01-03  1.801396e+11  1.924310e+11  1.772982e+11  1.901461e+11   
2020-01-04  1.902676e+11  1.924210e+11  1.878728e+11  1.909015e+11   
2020-01-05  1.910407e+11  1.953431e+11  1.895703e+11  1.913634e+11   

                                TOTAL3                              \
                  Volume          open          high           low   
Date                                                                 
2020-01-01  5.977114e+10  4.148759e+10  4.240917e+10  4.118580e+10   
2020-01-02  6.524542e+10  4.167834e+10  4.200154e+10  3.978805e+10   
2020-01-03  8.637843e+10  4.032308e+10  4.335032e+10  3.954749e+10   
2020-01-04  6.038307e+10  4.256472e+10  4.345286e+10  4.182455e+10   
2020-01-05  6.522888e+10  4.298122e+10  4.425466e+10  4.239119e+10   

                                          USDT_D                      \
                   close        Volume      open      high       low   
Date                                                                   
2020-01-01  4.176208e+10  3.320508e+10  2.208425  2.212275  2.182023   
2020-01-02  4.034357e+10  3.634974e+10  2.203964  2.285624  2.196909   
2020-01-03  4.247451e+10  4.791600e+10  2.273203  2.301038  2.134814   
2020-01-04  4.288096e+10  3.452323e+10  2.155914  2.180049  2.135203   
2020-01-05  4.319884e+10  3.799969e+10  2.145383  2.162024  2.104587   

                                    
               close        Volume  
Date                                
2020-01-01  2.201859  2.156231e+10  
2020-01-02  2.271307  2.431760e+10  
2020-01-03  2.155930  3.233531e+10  
2020-01-04  2.148195  2.154338e+10  
2020-01-05  2.142817  2.412223e+10